# LLM的记忆能力

## 1. SQLChatMessageHistory

用于连接关系性数据库，并返回一个可以**加载db数据** 和 **保存数据到db** 的对象。

```python
from langchain_community.chat_message_histories import SQLChatMessageHistory
```

参数：

- session_id: 数据的标识ID,session_id。
- connection_string: str, 连接db的字符串。
  - sqlite同步：sqlite:///./db/memory.db
  - sqlite异步：sqlite+aiosqlite:///./db/memory.db
  - mysql同步：mysql+pymysql://username:password@ip:3306/db_name
  - mysql异步：mysql+aiomysql://username:password@ip:3306/db_name
- connection: 数据库的连接对象，依赖sqlalchemy这个库来创建连接对象;推荐使用connection。
- async_mode: bool,默认为False，异步连接(对应ainvoke()/astream())的情况下，需要设置为True。

方法：

- add_message(): 将参数内容存入DB，参数为SystemMessage/HumanMessage/AIMessage。
- add_messages():将参数内容存入DB，参数为SystemMessage[]/HumanMessage[]/AIMessage[]。
- get_messages(): 获取db的数据。
- aadd_message(): 异步方式,将参数内容存入DB，参数为SystemMessage/HumanMessage/AIMessage。
- aadd_messages(): 异步方式,将参数内容存入DB，参数为SystemMessage[]/HumanMessage[]/AIMessage[]。
- aget_messages():异步方式,获取db的数据。


In [8]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

chat_history = SQLChatMessageHistory("daxiaohan_01", "sqlite:///./db/memory.db")

In [9]:
chat_history.add_message(SystemMessage(content="你是一名作家，叫做费小V"))
chat_history.add_messages([HumanMessage(content="我叫大小寒"), AIMessage(content="您好，大小寒，我记住你啦！")])

In [10]:
result = chat_history.get_messages()
result

[SystemMessage(content='你是一名作家，叫做费小V'),
 HumanMessage(content='我叫大小寒'),
 AIMessage(content='您好，大小寒，我记住你啦！')]

In [13]:
chat_history2 = SQLChatMessageHistory("daxiaohan_02", "sqlite:///./db/memory.db")
chat_history2.add_message(HumanMessage(content="xxxxx@@@@@@"))
result = chat_history2.get_messages()
result

[HumanMessage(content='xxxxx@@@@@@')]

In [2]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

chat_history = SQLChatMessageHistory("daxiaohan_01", "sqlite+aiosqlite:///./db/memory.db", async_mode=True)


In [9]:
result = await chat_history.aget_messages()
result

[SystemMessage(content='你是一名作家，叫做费小V'),
 HumanMessage(content='我叫大小寒'),
 AIMessage(content='您好，大小寒，我记住你啦！'),
 HumanMessage(content='6666')]

In [8]:
await chat_history.aadd_message(HumanMessage(content="6666"))

### 1.1 sqlalchemy

是python中应用广泛的一个ORM框架。ORM，Object-Relational-Mapping，对象关系映射。它将一个对象模型映射到关系数据库。

```python
from sqlalchemy import create_engine
from sqlalchemy.ext.asyncio import create_async_engine
```

- create_engine(database_url): 建立同步连接，参数为连接db的协议字符串，比如：sqlite:///./db/memory.db
- create_async_engine(database_url):  建立异步连接，参数为连接db的协议字符串，比如：sqlite+aiosqlite:///./db/memory.db


In [10]:
from sqlalchemy import create_engine
from sqlalchemy.ext.asyncio import create_async_engine


chat_history_03 = SQLChatMessageHistory("daxiaohan_02", connection=create_engine("sqlite:///./db/memory.db"))
chat_history_04 = SQLChatMessageHistory("daxiaohan_01", connection=create_async_engine("sqlite+aiosqlite:///./db/memory.db"), async_mode=True)



In [11]:
chat_history_03.get_messages()

[HumanMessage(content='xxxxx@@@@@@')]

In [12]:
result = await chat_history_04.aget_messages()
result

[SystemMessage(content='你是一名作家，叫做费小V'),
 HumanMessage(content='我叫大小寒'),
 AIMessage(content='您好，大小寒，我记住你啦！'),
 HumanMessage(content='6666')]

### 1.2 SQLite

#### (1) 安装

在linux和macos上已经预装了。

https://www.runoob.com/sqlite/sqlite-intro.html

#### (2) 

```shell
# 进入sqlite命令行,sqlite3命令后可以跟一个.db文件，也可以不跟
sqlite3 
# 退出sqlite命令行
> .quit/.exit
# 打开/创建db(没有会在当前目录创建一个db文件，如果已经存在则打开)
> .open test.db
# 查看有多少个db
> .databases
# 查看db中有多少张表
> .tables
```


# 2. RunnableWithMessageHistory

### 2.1 

用于为langchain加载历史记录信息，以及在模型返回后保存记录信息。

参数：

- runnable: langchain对象, chain = prompt | llm
- get_session_history(session_id: str): 这个参数接收一个方法，这个方法会接收到一个session_id,
  
  需要返回一个能够获 **加载db数据** 和 **保存数据到db** 的对象，比如说SQLChatMessageHistory对象
- history_messages_key: str, 字符串，指定加载的历史记录信息放到inputs字典的哪个key中。

### 2.2 MessagesPlaceholder

Message列表数据的占位符，用在prompt中。

```python
MessagesPlaceholder(variable_name="history")
```

### 2.3 RunnableConfig

invoke()/ainvoke()/stream()/astream()的第二个参数。

```json
    {"configurable": {"session_id": user_id}}
```